# Inaugural Project

Imports and set magics:

In [ ]:
import numpy as np
import matplotlib.pylab as plt
from scipy import optimize
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import inauguralproject

# Question 1

In [ ]:
#In this question, we want to construct a function which solves equation (1). 

#Firstly, the utility function is defined by: 
def utility(w, l, m = 1, v = 10, epsilon = 0.3, tau0 = 0.4, tau1 = 0.1, kappa = 0.4):
    tax = tau0*w*l+tau1*np.fmax(w*l-kappa,0)
    c = m+w*l-tax
    utility = np.log(c)-v*l**(1+1/epsilon)/(1+1/epsilon)
    return utility,c, tax

#Next a function is created in order to maximize utility:
def solveconsumerproblem(w, m = 1, v = 10, epsilon = 0.3, tau0 = 0.4, tau1 = 0.1, kappa = 0.4,\
                         N=100, callableoutput=False, tax=False):


#Now, we want to create a set of emply local lists in order to store the outputs. 
    c_star = [0]
    l_star = [0]
    t_star = [0]
    utility_star = [-np.inf]

# Creating a range for labour in order to examine the utility for a given number of elements. 
    Range = np.linspace(0, 1, N)
    for i in Range:
        utility_temp = utility(l = i, w = w, epsilon = epsilon, tau0 = tau0, tau1 = tau1, kappa = kappa)
        if utility_temp[0]>utility_star:
            utility_star[0]=utility_temp[0]
            l_star[0] = i
            c_star[0] = utility_temp[1]
            t_star[0] = utility_temp[2]

#Setting requirements for tax, whether to display or not
    if tax == False:
        if callableoutput == False:
            print(f"It is found that (l*,c*) = ({l_star[0]:.3}, {c_star[0]:.3}) meaning that the optimal utility is given by u(c*,l*) = {utility_star[0]:.3}")  
        else:
            return utility_star[0], l_star[0], c_star[0]
    
    if tax == True:
            return utility_star[0], l_star[0], c_star[0], t_star[0]

solveconsumerproblem(w=0.6, tax=False, callableoutput=False)

# Question 2

In [ ]:
#Now we want to plot l* and c* as functions of w in the range 0.5 to 1.5

#In order to do so, we first define the range of w as: 
w_range = np.linspace(0.5, 1.5, 200) 

#As before, we create empty lists for later use:
l_list = []
c_list = []

#Lastly, we loop the w_range as follows:
for i in w_range:
    l_list.append(solveconsumerproblem(w = i, callableoutput=True)[1])
    c_list.append(solveconsumerproblem(w = i, callableoutput=True)[2])

#Now, the figures are ready to be created. This is done by: 


##//fig = plt.figure(figsize=(5,5))

# Fiure 1
##ax = fig.add_subplot(2,1,1)
##plt.plot(w_range,l_list)
##ax.set_xlabel('Wage')
##ax.set_ylabel("Labour")


#Figure 2
##ax = fig.add_subplot(2,1,2)
##ax.plot(w_range, c_list)
##ax.set_xlabel('Wage')
##ax.set_ylabel('Consumption')
# plot the results///

fig = plt.figure(figsize=(30,7.5))
plt.style.use('seaborn-whitegrid')

# first plot including the labor supply depending on the wage
plot_left = fig.add_subplot(1,2,1)
plot_left.plot(w_range,l_list)
plot_left.set_title("Labor depending on wage",fontsize=20)
plot_left.set_xlabel("Wage",fontsize=15)
plot_left.set_ylabel("Labor supply",fontsize=15)
plot_left.axes.tick_params(labelsize=15)

# second plot including the consumptions depending on the wage
plot_right = fig.add_subplot(1,2,2) 
plot_right.plot(w_range,c_list)
plot_right.set_title("Consumption depending on the wage",fontsize=20)
plot_right.set_xlabel("Wage",fontsize=15)
plot_right.set_ylabel("Optimal consumption",fontsize=15)
plot_right.axes.tick_params(labelsize=15)

# Question 3

In [ ]:
#In order to calculate the tax revenue, we need to find the wage-level for different types of consumers. This is done by following:

np.random.seed(seed = 1337)
wage_list = np.random.uniform(low = 0.5, high = 1.5, size = 10000)

#Next, we wish to find the tax-revenue for a set of different wage-levels and parameters as:
def taxrev(wages, tau0, tau1, kappa, epsilon = 0.3):

#Again, an empty list is generated for output:
    tax_payments = []

#As seen before, we want to create a loop for wages, adding the indiviaual tax element in the function.
    for i in wages:
       tax_payments.append(solveconsumerproblem(w = i, tau0=tau0, tau1=tau1, epsilon=epsilon, kappa=kappa, callableoutput = True, tax = True)[-1]) 
    
#The sum of all calculated individual taxpayments are used in order to find the aggregated tax revenue:
    return np.sum(tax_payments)

#Last, we find the total tax revenue as:
print("The aggregated tax revenue is calculated as " + \
str(round(taxrev(wages = wage_list, tau0=0.4, tau1=0.1, kappa=0.4, epsilon=0.3),2)))

# Question 4

In [ ]:
#Now, we wish to examine the aggregated tax revenue for epsilon=0.1. This is solely done by running the above function, setting epsilon=0.1 as follows:

print("For epsilon=0.1, the aggregated tax revenue is " + \
str(round(taxrev(wages = wage_list, tau0=0.4, tau1=0.1, kappa=0.4, epsilon=0.1),2)))

# Question 5

ADD ANSWER.

In [30]:
#Now we wish to maximize the tax revenue. This is done by the Powell-method 

def negtaxrev(paraVec, wages = wage_list):
     'Returns the negative value of tax revenue'
    
    # We use a vector of the parameters to find the taxrev and multiply it by -1, so we can use a              minimizer.
     tau0 = paraVec[0]
     tau1 = paraVec[1]
     kappa = paraVec[2]
     temptaxrev = -1*taxrev(wages = wages, tau0 = tau0, tau1= tau1, kappa=kappa)

     # Band-aid solution to ensure kappa, the top tax cut-off, is non-negative.
     if kappa < 0:
          return 1
     else:
          return temptaxrev

initial_guess = [0.3, 0.3, 0.3]

# We use the scipy function, optimize.minimize to minimize negtaxrev. We use the Powell method, because some of the other methods didn't work or took suspiciously long time to run. Furthermore, since it isn't clearly specified wheter we should use epsilon = 0.1 or 0.3 in this question, we have chosen to use epsilon = 0.3.
sol_taxrev = optimize.minimize(negtaxrev, initial_guess, method = 'Powell')
print('Taxrex = ' + str(round(-sol_taxrev['fun'],3)))
print('Tau0 = '+ str(round(sol_taxrev['x'][0],3)) + ' Tau1 = '+ str(round(sol_taxrev['x'][1],3)) + \
' Kappa = '+ str((sol_taxrev['x'][2])))

# Conclusion

ADD CONCISE CONLUSION.